<div class="alert alert-block alert-warning">
<b>Warning:</b> Before running below cells please make sure you have API key and XYZ access token. 
                Please see <a href="https://github.com/heremaps/here-map-widget-for-jupyter/blob/master/examples/README.md">README.md</a> for more info on API key and XYZ access token.
</div>

In [ ]:
import os
os.environ["LS_API_KEY"] = "MY-API-KEY" # replace your API key here.
os.environ["XYZ_TOKEN"] = "MY-XYZ-TOKEN" # replace your XYZ access token here.

## Do all the required import

In [ ]:
from xyzspaces import XYZ
import os
import time

from functools import partial
import concurrent.futures
from multiprocessing import Process, Manager

## Generate a Data Hub object

In [ ]:
xyz_token = os.environ["XYZ_TOKEN"]
xyz = XYZ(credentials=xyz_token)

## Generate the required Data Hub space object

In [ ]:
space_forest_data = xyz.spaces.from_id('ZManj2D5')
space = xyz.spaces.from_id('9YfUmtzQ')

## Generate a Country Polygon Feature List from Data Hub Space

In [ ]:
from geojson import Feature, FeatureCollection

features_list = [] 
for feature in space.iter_feature():
    features_list.append(feature)

## Update each Country Polygon Feature with the Forest Fire Count in that Country

In [ ]:
manager = Manager()
country_forest_count_list = manager.list()

In [ ]:
from forest_fire_analysis import _get_count_features

In [ ]:
%%time
part_func = partial(
                _get_count_features,
                forest_space=space_forest_data,
                country_forest_count_list=country_forest_count_list
            )
with concurrent.futures.ProcessPoolExecutor() as executor:
    for ft in executor.map(part_func, features_list, chunksize=9):
        print(f"features processed: {ft}")

In [ ]:
country_forest_count = list(country_forest_count_list)

## Provide style property to each Country Polygon Feature based on the Forest Fire Count in that Country 

In [ ]:
count = []
for feature in country_forest_count:
    count.append(feature['properties']['fire_count'])

In [ ]:
from branca.colormap import linear

colormap = linear.YlOrRd_04


colormap = colormap.scale(min(count), max(count))

for feature in country_forest_count:
    feature['properties']['style'] = dict(
                fillColor=colormap(feature['properties']['fire_count'])[:7],
                color='black',
                weight=0.9
            )

## Example of a Country Polygon Feature after processing

In [ ]:
country_forest_count[0]

## Plot the Country Polygon Features as Feature Collection on Here Map Widget

In [ ]:
from here_map_widget import Map
from here_map_widget import GeoJSON
import os

m = Map(api_key=os.environ['LS_API_KEY'])
geojson = GeoJSON(data=FeatureCollection(country_forest_count), disable_legacy_mode=True)
m.add_layer(geojson)
m